In [1]:
import os
import sys
import numpy as np

from nltk.classify import NaiveBayesClassifier
import nltk

In [3]:
!head data/ratings_train.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


In [5]:
!head data/ratings_test.txt

id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


In [6]:
TRAIN_FILE = 'data/ratings_train.txt'
TEST_FILE = 'data/ratings_test.txt'

In [9]:
# Read data
train_reviews = []
with open(TRAIN_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, label = line.strip().split('\t')
        train_reviews.append((review.strip(), label.strip()))
        
test_reviews = []        
with open(TEST_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, label = line.strip().split('\t')
        test_reviews.append((review.strip(), label.strip()))

In [10]:
train_reviews[:10]

[('아 더빙.. 진짜 짜증나네요 목소리', '0'),
 ('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'),
 ('너무재밓었다그래서보는것을추천한다', '0'),
 ('교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'),
 ('사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'),
 ('막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'),
 ('원작의 긴장감을 제대로 살려내지못했다.', '0'),
 ('별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'),
 ('액션이 없는데도 재미 있는 몇안되는 영화', '1'),
 ('왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1')]

In [11]:
test_reviews[:10]

[('굳 ㅋ', '1'),
 ('GDNTOPCLASSINTHECLUB', '0'),
 ('뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '0'),
 ('지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0'),
 ('3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', '0'),
 ('음악이 주가 된, 최고의 음악영화', '1'),
 ('진정한 쓰레기', '0'),
 ('마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다', '0'),
 ('갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
  '0'),
 ('이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..', '1')]

In [12]:
from konlpy.tag import Twitter
twi_tagger = Twitter()

In [13]:
def tokenizer(doc):
    tokens = ['/'.join(t) for t in twi_tagger.pos(doc, norm=True, stem=True)]
    return tokens

In [15]:
%%time
train_reviews = [(tokenizer(review), label) for review, label in train_reviews]
test_reviews = [(tokenizer(review), label) for review, label in test_reviews]

CPU times: user 6min 24s, sys: 3.63 s, total: 6min 28s
Wall time: 7min 20s


In [16]:
train_reviews[:5]

[(['아/Exclamation',
   '더빙/Noun',
   '../Punctuation',
   '진짜/Noun',
   '짜증/Noun',
   '나다/Verb',
   '목소리/Noun'],
  '0'),
 (['흠/Noun',
   '.../Punctuation',
   '포스터/Noun',
   '보고/Noun',
   '초딩/Noun',
   '영화/Noun',
   '줄/Noun',
   '..../Punctuation',
   '오버/Noun',
   '연기/Noun',
   '조차/Josa',
   '가볍다/Adjective',
   '않다/Verb'],
  '1'),
 (['너무/Noun',
   '재/Noun',
   '밓었/Noun',
   '다/Josa',
   '그래서/Adverb',
   '보다/Verb',
   '추천/Noun',
   '하다/Verb'],
  '0'),
 (['교도소/Noun',
   '이야기/Noun',
   '구먼/Noun',
   '../Punctuation',
   '솔직하다/Adjective',
   '재미/Noun',
   '는/Josa',
   '없다/Adjective',
   '../Punctuation',
   '평점/Noun',
   '조정/Noun'],
  '0'),
 (['사이몬페그/Noun',
   '의/Josa',
   '익살/Noun',
   '스런/Josa',
   '연기/Noun',
   '가/Josa',
   '돋보이다/Verb',
   '영화/Noun',
   '!/Punctuation',
   '스파이더맨/Noun',
   '에서/Josa',
   '늙다/Verb',
   '보이다/Verb',
   '하다/Verb',
   '크다/Verb',
   '스틴/Noun',
   '던스트/Noun',
   '가/Josa',
   '너무나/Noun',
   '도/Josa',
   '이쁘다/Adjective',
   '보이다/Verb'],
  '1')]

In [17]:
def term_exists(doc):
    return {word: True for word in doc}

In [18]:
train_xy = [(term_exists(d), c) for d, c in train_reviews] 
test_xy = [(term_exists(d), c) for d, c in test_reviews] 

In [19]:
train_xy[:5]

[({'../Punctuation': True,
   '나다/Verb': True,
   '더빙/Noun': True,
   '목소리/Noun': True,
   '아/Exclamation': True,
   '진짜/Noun': True,
   '짜증/Noun': True},
  '0'),
 ({'..../Punctuation': True,
   '.../Punctuation': True,
   '가볍다/Adjective': True,
   '보고/Noun': True,
   '않다/Verb': True,
   '연기/Noun': True,
   '영화/Noun': True,
   '오버/Noun': True,
   '조차/Josa': True,
   '줄/Noun': True,
   '초딩/Noun': True,
   '포스터/Noun': True,
   '흠/Noun': True},
  '1'),
 ({'그래서/Adverb': True,
   '너무/Noun': True,
   '다/Josa': True,
   '밓었/Noun': True,
   '보다/Verb': True,
   '재/Noun': True,
   '추천/Noun': True,
   '하다/Verb': True},
  '0'),
 ({'../Punctuation': True,
   '교도소/Noun': True,
   '구먼/Noun': True,
   '는/Josa': True,
   '솔직하다/Adjective': True,
   '없다/Adjective': True,
   '이야기/Noun': True,
   '재미/Noun': True,
   '조정/Noun': True,
   '평점/Noun': True},
  '0'),
 ({'!/Punctuation': True,
   '가/Josa': True,
   '너무나/Noun': True,
   '늙다/Verb': True,
   '던스트/Noun': True,
   '도/Josa': True,
   '돋보이다/Verb': True,

In [20]:
test_xy[:5]

[({'ㅋ/KoreanParticle': True, '굳다/Adjective': True}, '1'),
 ({'GDNTOPCLASSINTHECLUB/Alpha': True}, '0'),
 ({'..../Punctuation': True,
   '10/Number': True,
   '나쁘다/Adjective': True,
   '는/Josa': True,
   '더/Noun': True,
   '더욱/Noun': True,
   '들/Suffix': True,
   '리/Noun': True,
   '뭐/Noun': True,
   '아니다/Adjective': True,
   '않다/Verb': True,
   '야/Josa': True,
   '은/Josa': True,
   '이/Noun': True,
   '점/Noun': True,
   '짜다/Verb': True,
   '평점/Noun': True},
  '0'),
 ({'..../Punctuation': True,
   '.../Punctuation': True,
   '돈/Noun': True,
   '막장/Noun': True,
   '보기/Noun': True,
   '않다/Verb': True,
   '에는/Josa': True,
   '완전/Noun': True,
   '임/Noun': True,
   '주다/Verb': True,
   '지루하다/Adjective': True},
  '0'),
 ({'../Punctuation': True,
   '3/Number': True,
   '??/Punctuation': True,
   'D/Alpha': True,
   '개/Noun': True,
   '나오다/Verb': True,
   '다섯/Noun': True,
   '로/Noun': True,
   '를/Josa': True,
   '만/Noun': True,
   '별/Noun': True,
   '불편하다/Adjective': True,
   '심기/Noun': True,
  

In [21]:
%%time
classifier = NaiveBayesClassifier.train(train_xy)

CPU times: user 3.96 s, sys: 61.8 ms, total: 4.02 s
Wall time: 4.03 s


In [24]:
# Accuracy
nltk.classify.util.accuracy(classifier, test_xy)

0.83504

In [23]:
classifier.show_most_informative_features()

Most Informative Features
                 최악/Noun = True                0 : 1      =     68.0 : 1.0
               울컥/Adverb = True                1 : 0      =     58.6 : 1.0
               ♥/Foreign = True                1 : 0      =     57.1 : 1.0
                 낚였/Noun = True                0 : 1      =     48.8 : 1.0
              Good/Alpha = True                1 : 0      =     48.6 : 1.0
                 낭비/Noun = True                0 : 1      =     47.4 : 1.0
                 펑펑/Noun = True                1 : 0      =     46.1 : 1.0
                 또봐/Noun = True                1 : 0      =     42.5 : 1.0
                 반개/Noun = True                0 : 1      =     41.1 : 1.0
                개그맨/Noun = True                0 : 1      =     39.5 : 1.0
